In [2]:
# Import libraries:
import pandas as pd
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
else:
    path = 'G:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7)
# Check for cancelled procedures:
# df_ids7 = bh_utils.filter_cancelled(df_ids7)
# Check for non-human subjects (e.g. phantoms, animals, etc.):
# df_ids7 = bh_utils.filter_phantom_etc(df_ids7)

# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7)
# Check whether an accession number is in the DoseTrack data:
df_ids7 = bh_utils.check_accession_ids7_vs_dt(df_ids7, df_dt)

Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2465
Number of accession numbers in IDS7 not in DoseTrack: 441


One patient can have the same accession no on different laboratories, due to erroneous registration in PACS.
If so please check the Modality/Room in DoseTrack.

If there is two different accession numbers at the same starting time they are likely the same examination.
There was an example of a patient with as little as 1 hr 22 min between examinations.
The Time of the procedure in dosetrack is the start of the procedure.


If there is two different accession numbers with the exact same time they are likely the same examination. 

We need to create logic for merging lines in the ids7 data:

During a booking the patient gets an accession number:
As the procedure moves forward more codes can be added to the same accession number. These codes should be merged into one line.

Check if the accession number exists in Dosetrack. If it does, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does not have a corresponding entry in dosetrack. If so they might be in the same procedure.

Alternatively, if there are an accession number that is not included in doseTrack, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does have a corresponding entry in dosetrack. If so they might be in the same procedure.

If neither of the procedures exists in dosetrack they might or might not be the same procedure.

Print a list of all the changes performed on the data.

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [ ]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)